### Selected Word Group In Youtube Sentence

In [1]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish Dutch"

# parameter
file = "Turkish Dutch SV"  # Turkish Dutch SV
sheets = "2 gram hybrid"  # 2 gram target, 2 gram hybrid
start_first_column = 0  # start column location number
time_shift = 0.6
sample_num = 6

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/3-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [7]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [8]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [9]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,twogram_pair1
0,ajan filmleri
1,alarm şifresi
2,alerjik şok
3,amatör radyo
4,arkeoloji profesörü
...,...
158,vejetaryen pizza
159,video mesajı
160,video sistemi
161,video ünitesi


In [10]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[start_first_column:]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
        else:
            pass

In [11]:
df_search_result = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,bebek fotoğrafları,457.180,460.780,kedi fotoğrafları bitti şimdide bebek fotoğraf...,B3ZakJ4O5uw
1,çikolata fabrikası,569.220,571.460,aynı charlienin çikolata fabrikası gibi,4ZzvROhMfjg
2,çikolata fabrikası,55.120,57.780,charlienin çikolata fabrikası filmini izledim ...,ZLpvMS78whQ
3,çikolata fabrikası,3574.893,3577.353,gerçi dedem bana çikolata fabrikası alır ama n...,mQUinrfsVkM
4,çikolatalı pasta,2444.100,2446.491,ama ben çikolatalı pasta istemedim ki soda ist...,0AOYUEuSiyw
...,...,...,...,...,...
67,teknik direktör,106.000,110.900,kloppun liverpooldaki en sevilen teknik direkt...,J0hq13epy2k
68,teknik direktör,814.094,819.094,ama teknik direktör holger osieck tanjuyu takı...,lOa7Qp9JpWA
69,teknik direktör,1186.000,1206.000,şimdi galatasaray ile ilgili birkaç sorum var ...,qzLawMz8ZG4
70,televizyon ekranı,327.460,329.820,hatırlarsan televizyon ekranı karşısında,eec5mG_90bk


In [12]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,bebek fotoğrafları,458.668000,459.628000,kedi fotoğrafları bitti şimdide bebek fotoğraf...,B3ZakJ4O5uw
1,çikolata fabrikası,570.081538,571.230256,aynı charlienin çikolata fabrikası gibi,4ZzvROhMfjg
2,çikolata fabrikası,55.542222,56.386667,charlienin çikolata fabrikası filmini izledim ...,ZLpvMS78whQ
3,çikolata fabrikası,3575.680200,3576.664200,gerçi dedem bana çikolata fabrikası alır ama n...,mQUinrfsVkM
4,çikolatalı pasta,2444.434740,2445.295500,ama ben çikolatalı pasta istemedim ki soda ist...,0AOYUEuSiyw
...,...,...,...,...,...
67,teknik direktör,107.844706,108.824706,kloppun liverpooldaki en sevilen teknik direkt...,J0hq13epy2k
68,teknik direktör,814.328375,815.656500,ama teknik direktör holger osieck tanjuyu takı...,lOa7Qp9JpWA
69,teknik direktör,1195.537037,1197.111111,şimdi galatasaray ile ilgili birkaç sorum var ...,qzLawMz8ZG4
70,televizyon ekranı,328.109000,329.230000,hatırlarsan televizyon ekranı karşısında,eec5mG_90bk


In [13]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,bebek fotoğrafları,458.068000,460.228000,kedi fotoğrafları bitti şimdide bebek fotoğraf...,B3ZakJ4O5uw
1,çikolata fabrikası,569.481538,571.830256,aynı charlienin çikolata fabrikası gibi,4ZzvROhMfjg
2,çikolata fabrikası,54.942222,56.986667,charlienin çikolata fabrikası filmini izledim ...,ZLpvMS78whQ
3,çikolata fabrikası,3575.080200,3577.264200,gerçi dedem bana çikolata fabrikası alır ama n...,mQUinrfsVkM
4,çikolatalı pasta,2443.834740,2445.895500,ama ben çikolatalı pasta istemedim ki soda ist...,0AOYUEuSiyw
...,...,...,...,...,...
67,teknik direktör,107.244706,109.424706,kloppun liverpooldaki en sevilen teknik direkt...,J0hq13epy2k
68,teknik direktör,813.728375,816.256500,ama teknik direktör holger osieck tanjuyu takı...,lOa7Qp9JpWA
69,teknik direktör,1194.937037,1197.711111,şimdi galatasaray ile ilgili birkaç sorum var ...,qzLawMz8ZG4
70,televizyon ekranı,327.509000,329.830000,hatırlarsan televizyon ekranı karşısında,eec5mG_90bk


In [14]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,bebek fotoğrafları,458,460,kedi fotoğrafları bitti şimdide bebek fotoğraf...,B3ZakJ4O5uw
1,çikolata fabrikası,569,572,aynı charlienin çikolata fabrikası gibi,4ZzvROhMfjg
2,çikolata fabrikası,55,57,charlienin çikolata fabrikası filmini izledim ...,ZLpvMS78whQ
3,çikolata fabrikası,3575,3577,gerçi dedem bana çikolata fabrikası alır ama n...,mQUinrfsVkM
4,çikolatalı pasta,2444,2446,ama ben çikolatalı pasta istemedim ki soda ist...,0AOYUEuSiyw
...,...,...,...,...,...
67,teknik direktör,107,109,kloppun liverpooldaki en sevilen teknik direkt...,J0hq13epy2k
68,teknik direktör,814,816,ama teknik direktör holger osieck tanjuyu takı...,lOa7Qp9JpWA
69,teknik direktör,1195,1198,şimdi galatasaray ile ilgili birkaç sorum var ...,qzLawMz8ZG4
70,televizyon ekranı,328,330,hatırlarsan televizyon ekranı karşısında,eec5mG_90bk


In [15]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,bebek fotoğrafları,458,460,kedi fotoğrafları bitti şimdide bebek fotoğraf...,B3ZakJ4O5uw,https://www.youtube.com/watch?v=B3ZakJ4O5uw&t=...
1,çikolata fabrikası,569,572,aynı charlienin çikolata fabrikası gibi,4ZzvROhMfjg,https://www.youtube.com/watch?v=4ZzvROhMfjg&t=...
2,çikolata fabrikası,55,57,charlienin çikolata fabrikası filmini izledim ...,ZLpvMS78whQ,https://www.youtube.com/watch?v=ZLpvMS78whQ&t=55s
3,çikolata fabrikası,3575,3577,gerçi dedem bana çikolata fabrikası alır ama n...,mQUinrfsVkM,https://www.youtube.com/watch?v=mQUinrfsVkM&t=...
4,çikolatalı pasta,2444,2446,ama ben çikolatalı pasta istemedim ki soda ist...,0AOYUEuSiyw,https://www.youtube.com/watch?v=0AOYUEuSiyw&t=...
...,...,...,...,...,...,...
67,teknik direktör,107,109,kloppun liverpooldaki en sevilen teknik direkt...,J0hq13epy2k,https://www.youtube.com/watch?v=J0hq13epy2k&t=...
68,teknik direktör,814,816,ama teknik direktör holger osieck tanjuyu takı...,lOa7Qp9JpWA,https://www.youtube.com/watch?v=lOa7Qp9JpWA&t=...
69,teknik direktör,1195,1198,şimdi galatasaray ile ilgili birkaç sorum var ...,qzLawMz8ZG4,https://www.youtube.com/watch?v=qzLawMz8ZG4&t=...
70,televizyon ekranı,328,330,hatırlarsan televizyon ekranı karşısında,eec5mG_90bk,https://www.youtube.com/watch?v=eec5mG_90bk&t=...


In [16]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_{sheets.capitalize()}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

#### Copy Move And Delete

In [36]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx")
output_file

['Turkish_Dutch_2 gram hybrid_6_Youtube_0.6s_Timeshift_Result.xlsx']

In [37]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [38]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass